# Pipeline Example performing the Bert Preprocessing with TensorFlow Transform

TFX + BERT Blog Post:  https://blog.tensorflow.org/2020/m03/part-1-fast-scalable-and-accurate-nlp-tensorflow-deploying-bert.html

Based on this example:  https://colab.research.google.com/github/tensorflow/workshops/blob/master/blog/TFX_Pipeline_for_Bert_Preprocessing.ipynb#scrollTo=ApwndjCnU2zt

## Motivation

Instead of converting the input to a tranformer model into token ids on the client side, the model exported from this pipeline will allow the conversion on the server side.

The pipeline takes advantage of the broad TensorFlow Eco system, including:
* Loading the IMDB dataset via **TensorFlow Datasets**
* Loading a pre-trained model via **tf.hub**
* Manipulating the raw input data with **tf.text**
* Building a simple model architecture with **Keras**
* Composing the model pipeline with **TensorFlow Extended**, e.g. TensorFlow Transform, TensorFlow Data Validation  

The structure of the overall pipeline follows the [TFX Taxi Cab example](https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/components.ipynb) 

### Outline

* Install Required Packages
* Load the training data set
* Set up helper functions, if running on TF 2.0
* Create the TFX Pipeline
* Export the trained Model
* Test the exported Model


# Project Setup

## Install Required Packages

**Note** 
After installing the required Python packages, you'll need to restart the Colab Runtime Engine (Menu > Runtime > Restart runtime...)

![Restart of the Colab Runtime Engine](https://drive.google.com/uc?id=1xnjAy2sxIymKhydkqb0RKzgVK9rh3teH)



In [1]:
!pip install -Uq wrapt --upgrade --ignore-installed
!pip install -Uq pip --upgrade
!pip install -Uq tensorflow==2.0.0
!pip install -Uq pyarrow==0.15.1
!pip install -Uq tfx
!pip install -Uq tensorflow-text
!pip install -Uq grpcio --upgrade

ERROR: tensorflow-text 2.1.1 has requirement tensorflow<2.2,>=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-serving-api 2.1.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: ml-metadata 0.21.2 has requirement tensorflow!=2.0.*,<3,>=1.15, but you'll have tensorflow 2.0.0 which is incompatible.


In [2]:
!pip list

Package                            Version   
---------------------------------- ----------
absl-py                            0.8.1     
alabaster                          0.7.10    
anaconda-client                    1.6.14    
anaconda-project                   0.8.2     
apache-beam                        2.17.0    
asn1crypto                         0.24.0    
astor                              0.8.1     
astroid                            1.6.3     
astropy                            3.0.2     
attrs                              18.1.0    
Automat                            0.3.0     
autovizwidget                      0.15.0    
avro-python3                       1.9.2.1   
awscli                             1.18.24   
Babel                              2.5.3     
backcall                           0.1.0     
backports.shutil-get-terminal-size 1.0.0     
bcrypt                             3.1.7     
beautifulsoup4                     4.6.0     
bitarray                          

In [3]:
!pip install -Uq tensorflow_hub
!pip install -Uq tensorflow_datasets

## Import relevant packages

In [4]:
import glob
import os
import pprint
import re
import tempfile
from shutil import rmtree

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_data_validation as tfdv
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_text as text
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.tf_metadata import (dataset_metadata, dataset_schema,
                                              metadata_io, schema_utils)

from tfx.components import ResolverNode
from tfx.components import ImportExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.proto import evaluator_pb2, example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## Check GPU Availability

Check if your Colab notebook is configured to use Graphical Processing Units (GPUs). If zero GPUs are available, check if the Colab notebook is configured to use GPUs (Menu > Runtime > Change Runtime Type).

![Hardware Accelerator Settings](https://drive.google.com/uc?id=1qrihuuMtvzXJHiRV8M7RngbxFYipXKQx)


In [5]:
num_gpus_available = 0
print("Num GPUs Available: ", num_gpus_available)

Num GPUs Available:  0


## Download the IMDB Dataset from TensorFlow Datasets

For our demo example, we are using the [IMDB data set](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) to train a sentiment model based on the pre-trained BERT model. The data set is provided through [TensorFlow Datasets](https://www.tensorflow.org/datasets). Our ML pipeline can read TFRecords, however it expects only TFRecord files in the data folder. That is the reason why we need to delete the additional files provided by TFDS.

In [10]:
!mkdir -p ./content/tfds/

In [11]:
def clean_before_download(base_data_dir):
    rmtree(base_data_dir)
    
def delete_unnecessary_files(base_path):
    os.remove(base_path + "dataset_info.json")
    os.remove(base_path + "label.labels.txt")
    
    counter = 2
    for f in glob.glob(base_path + "imdb_reviews-unsupervised.*"):
        os.remove(f)
        counter += 1
    print(f"Deleted {counter} files")

def get_dataset(name='imdb_reviews', version="1.0.0"):

    base_data_dir = "./content/tfds/"
    config="plain_text"
    version="1.0.0"

    clean_before_download(base_data_dir)
    tfds.disable_progress_bar()
    builder = tfds.text.IMDBReviews(data_dir=base_data_dir, 
                                    config=config, 
                                    version=version)
    download_config = tfds.download.DownloadConfig(
        download_mode=tfds.GenerateMode.FORCE_REDOWNLOAD)
    builder.download_and_prepare(download_config=download_config)

    base_tfrecords_filename = os.path.join(base_data_dir, "imdb_reviews", config, version, "")
    train_tfrecords_filename = base_tfrecords_filename + "imdb_reviews-train*"
    test_tfrecords_filename = base_tfrecords_filename + "imdb_reviews-test*"
    label_filename = os.path.join(base_tfrecords_filename, "label.labels.txt")
    labels = [label.rstrip('\n') for label in open(label_filename)]
    delete_unnecessary_files(base_tfrecords_filename)
    return (train_tfrecords_filename, test_tfrecords_filename), labels

tfrecords_filenames, labels = get_dataset()

Shuffling and writing examples to ./content/tfds/imdb_reviews/plain_text/1.0.0.incompleteSHFCNM/imdb_reviews-train.tfrecord
Shuffling and writing examples to ./content/tfds/imdb_reviews/plain_text/1.0.0.incompleteSHFCNM/imdb_reviews-test.tfrecord
Shuffling and writing examples to ./content/tfds/imdb_reviews/plain_text/1.0.0.incompleteSHFCNM/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to ./content/tfds/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Deleted 3 files


# Model Architecture for our Sentiment Classification

In our ML pipeline we need to use TensorFlow Estimators (TFX currently doesn't support the Keras). However, that doesn't stop use from defining the model with Keras and then converting it to a TensorFlow Estimator. 

In [12]:
%%skip_for_export
%%writefile bert.py

import tensorflow_hub as hub

BERT_TFHUB_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"

def load_bert_layer(model_url=BERT_TFHUB_URL):
    # Load the pre-trained BERT model as layer in Keras
    bert_layer = hub.KerasLayer(
        handle=model_url,
        trainable=True)
    return bert_layer


Overwriting bert.py
This cell will be skipped during export to pipeline.


## TensorFlow 2.0 Helper Functions

In [13]:
%%skip_for_export
%%writefile tf2_0_helpers.py

"""
If you need to run this ML pipeline on <TF2.1rc0 and >TF1.X, load the 
tokenization table with the code provided below   

The _CreateTable functions are redundant with the release of 
https://github.com/tensorflow/text/commit/bc7c213fb89d0bd323b6b674b15f2f6be4bf1500
"""

import tensorflow as tf

def merge_dims(rt, axis=0):
    """
    Function was suggested in this Github issue:
    https://github.com/tensorflow/text/issues/155
    
    Collapses the specified axis of a RaggedTensor.

    Suppose we have a RaggedTensor like this:
    [[1, 2, 3],
    [4, 5],
    [6]]

    If we flatten the 0th dimension, it becomes:
    [1, 2, 3, 4, 5, 6]

    Args:
      rt: a RaggedTensor.
      axis: the dimension to flatten.

    Returns:
      A flattened RaggedTensor, which now has one less dimension.
    """
    to_expand = rt.nested_row_lengths()[axis]
    to_elim = rt.nested_row_lengths()[axis + 1]

    bar = tf.RaggedTensor.from_row_lengths(to_elim, row_lengths=to_expand)
    new_row_lengths = tf.reduce_sum(bar, axis=1)
    return tf.RaggedTensor.from_nested_row_lengths(
        rt.flat_values,
        rt.nested_row_lengths()[:axis] + (new_row_lengths,))

def load_vocab_data(vocab_file):
    vocab_list = []
    with open(vocab_file, "r") as f:
        for i in f.readlines():
            vocab_list.append(i.strip('\n'))
    return vocab_list

def _CreateTable(num_oov=1):

    vocab = load_vocab_data()
    init = tf.lookup.KeyValueTensorInitializer(
        vocab,
        tf.range(tf.size(vocab, out_type=tf.int64), dtype=tf.int64),
        key_dtype=tf.string,
        value_dtype=tf.int64)
    return tf.lookup.StaticVocabularyTable(
        init, num_oov, lookup_key_dtype=tf.string)

Overwriting tf2_0_helpers.py
This cell will be skipped during export to pipeline.


# TFX Pipeline

The TensorFlow Extended Pipeline is more or less following the example setup shown here. We'll only note deviations from the original setup.

## Initializing the Interactive TFX Pipeline

In [14]:
context = InteractiveContext()

## Loading the dataset

In [15]:
!ls -al ./content/tfds/imdb_reviews/plain_text/1.0.0

total 66548
drwxr-xr-x 2 ec2-user ec2-user     4096 Apr 20 00:49 .
drwxr-xr-x 3 ec2-user ec2-user     4096 Apr 20 00:49 ..
-rw-rw-r-- 1 ec2-user ec2-user 33675595 Apr 20 00:49 imdb_reviews-test.tfrecord-00000-of-00001
-rw-rw-r-- 1 ec2-user ec2-user 34457750 Apr 20 00:48 imdb_reviews-train.tfrecord-00000-of-00001


In [16]:
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=45),
                 example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=5)
             ]))
# Load the data from our prepared TFDS folder
examples = external_input("./content/tfds/imdb_reviews/plain_text/1.0.0")
example_gen = ImportExampleGen(input=examples, output_config=output)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/ImportExampleGen/examples/1, id: 2)]
        ))

In [17]:
%%skip_for_export

for artifact in example_gen.outputs['examples'].get():
    print(artifact.uri)

/tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/ImportExampleGen/examples/1
This cell will be skipped during export to pipeline.


## TensorFlow Data Validation

In [18]:
%%skip_for_export

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


In [19]:
%%skip_for_export

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'text',BYTES,required,,-
'label',INT,required,,-


This cell will be skipped during export to pipeline.


In [20]:
%%skip_for_export

# check the data schema for the type of input tensors
tfdv.load_schema_text(schema_gen.outputs['schema'].get()[0].uri + "/schema.pbtxt")

feature {
  name: "text"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "label"
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}

This cell will be skipped during export to pipeline.


In [21]:
%%skip_for_export

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

This cell will be skipped during export to pipeline.


## TensorFlow Transform

This is where we perform the BERT processing. 


In [22]:
%%skip_for_export
%%writefile transform.py

import tensorflow as tf
import tensorflow_text as text

from bert import load_bert_layer
# from tf2_0_helpers import merge_dim, _CreateTable, DEFAULT_VOCAB_FILE  # if TF2.0

MAX_SEQ_LEN = 64  # max number is 512
do_lower_case = load_bert_layer().resolved_object.do_lower_case.numpy()

def preprocessing_fn(inputs):
    """Preprocess input column of text into transformed columns of.
        * input token ids
        * input mask
        * input type ids
    """

    CLS_ID = tf.constant(101, dtype=tf.int64)
    SEP_ID = tf.constant(102, dtype=tf.int64)
    PAD_ID = tf.constant(0, dtype=tf.int64)

    # vocab_lookup_table = _CreateTable()  # only need for < TF2.1rc0
    vocab_file_path = load_bert_layer().resolved_object.vocab_file.asset_path
    
    bert_tokenizer = text.BertTokenizer(vocab_lookup_table=vocab_file_path, 
                                        token_out_type=tf.int64, 
                                        lower_case=do_lower_case) 
    
    def tokenize_text(text, sequence_length=MAX_SEQ_LEN):
        """
        Perform the BERT preprocessing from text -> input token ids
        """

        # convert text into token ids
        tokens = bert_tokenizer.tokenize(text)
        
        # flatten the output ragged tensors 
        tokens = tokens.merge_dims(1, 2)[:, :sequence_length]
        
        # Add start and end token ids to the id sequence
        start_tokens = tf.fill([tf.shape(text)[0], 1], CLS_ID)
        end_tokens = tf.fill([tf.shape(text)[0], 1], SEP_ID)
        tokens = tokens[:, :sequence_length - 2]
        tokens = tf.concat([start_tokens, tokens, end_tokens], axis=1)

        # truncate sequences greater than MAX_SEQ_LEN
        tokens = tokens[:, :sequence_length]

        # pad shorter sequences with the pad token id
        tokens = tokens.to_tensor(default_value=PAD_ID)
        pad = sequence_length - tf.shape(tokens)[1]
        tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)

        # and finally reshape the word token ids to fit the output 
        # data structure of TFT  
        return tf.reshape(tokens, [-1, sequence_length])

    def preprocess_bert_input(text):
        """
        Convert input text into the input_word_ids, input_mask, input_type_ids
        """
        input_word_ids = tokenize_text(text)
        input_mask = tf.cast(input_word_ids > 0, tf.int64)
        input_mask = tf.reshape(input_mask, [-1, MAX_SEQ_LEN])
        
        zeros_dims = tf.stack(tf.shape(input_mask))
        input_type_ids = tf.fill(zeros_dims, 0)
        input_type_ids = tf.cast(input_type_ids, tf.int64)

        return (
            input_word_ids, 
            input_mask,
            input_type_ids
        )

    input_word_ids, input_mask, input_type_ids = \
        preprocess_bert_input(tf.squeeze(inputs['text'], axis=1))

    return {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids,
        'label': inputs['label']
    }

Overwriting transform.py
This cell will be skipped during export to pipeline.


In [23]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Transform/transform_graph/5/.temp_path/tftransform_tmp/913569629e52494d87f0ba9c519cdc68/assets
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'list' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Transform/transform_graph/5/.temp_path/tftransform_tmp/913569629e52494d87f0ba9c519cdc68/saved_model.pb
va

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Transform/transformed_examples/5, id: 7)]
        ))

#### Check the Output Data Struture of the TF Transform Operation

In [24]:
from tfx_bsl.coders.example_coder import ExampleToNumpyDict

pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[0].uri

print(train_uri)

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_folders = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
tfrecord_filenames = []
for tfrecord_folder in tfrecord_folders:
    for name in os.listdir(tfrecord_folder):
        tfrecord_filenames.append(os.path.join(tfrecord_folder, name))


# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = ExampleToNumpyDict(serialized_example)
    pp.pprint(example)

/tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Transform/transformed_examples/5
{'input_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_type_ids': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_word_ids': array([  101,  2004,  1037,  2280, 26809,  3076,  1045,  5632,  2023,
        2143,  2200,  2172,  1012,  2009,  2001,  2061, 12689,  1998,
        6057,  1012,  2009,  2428,  3856,  2039,  1996,  4382,  2008,
        6526,  2426, 26809,  2493,  1012,  1045,  3246,  1010,  2116,
        2060,  2493,  2097,  3582,  2023,  3325,  1010,  2205,  1012,
        2174,  1010,  1045,  4687,  2065,  2023,  3185,  2003,  2035,
       

## Train of the Keras Model

In [25]:
%%skip_for_export
%%writefile trainer.py

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from typing import Text

import absl
import tensorflow as tf
from tensorflow import keras
import tensorflow_transform as tft
from tfx.components.trainer.executor import TrainerFnArgs


_LABEL_KEY = 'label'
BERT_TFHUB_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def load_bert_layer(model_url=BERT_TFHUB_URL):
    # Load the pre-trained BERT model as layer in Keras
    bert_layer = hub.KerasLayer(
        handle=model_url,
        trainable=True)
    return bert_layer

def get_model(tf_transform_output, max_seq_length=64, num_labels=2):

    # dynamically create inputs for all outputs of our transform graph
    feature_spec = tf_transform_output.transformed_feature_spec()  
    # the feature_spec contains the output feature `label` as well, we need to 
    # remove that feature column
    feature_spec.pop(_LABEL_KEY)

    inputs = {
        key: tf.keras.layers.Input(shape=(max_seq_length), name=key, dtype=tf.int64)
            for key in feature_spec.keys()
    }

    input_word_ids = tf.cast(inputs["input_word_ids"], dtype=tf.int32)
    input_mask = tf.cast(inputs["input_mask"], dtype=tf.int32)
    input_type_ids = tf.cast(inputs["input_type_ids"], dtype=tf.int32)

    # input_word_ids = tf.cast(input_word_ids, dtype=tf.int32)
    # input_mask = tf.cast(input_mask, dtype=tf.int32)
    # input_type_ids = tf.cast(input_type_ids, dtype=tf.int32)

    bert_layer = load_bert_layer()
    pooled_output, sequence_output = bert_layer(
        [input_word_ids, 
         input_mask, 
         input_type_ids
        ]
    )
    
    # Add additional layers depending on your problem
    x = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
    dense = tf.keras.layers.Dense(64, activation='relu')(x)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    keras_model = tf.keras.Model(
        inputs=[
                inputs['input_word_ids'], 
                inputs['input_mask'], 
                inputs['input_type_ids']], 
        outputs=pred)
    keras_model.compile(loss='binary_crossentropy', 
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                        metrics=['accuracy']
                        )
    return keras_model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)
        transformed_features.pop(_LABEL_KEY)

        outputs = model(transformed_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn

def _input_fn(file_pattern: Text,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 32) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
      file_pattern: input tfrecord file pattern.
      tf_transform_output: A TFTransformOutput.
      batch_size: representing the number of consecutive elements of returned
        dataset to combine in a single batch

    Returns:
      A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=_LABEL_KEY)

    return dataset

# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.

    Args:
      fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 32)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 32)

    # mirrored_strategy = tf.distribute.MirroredStrategy()
    # with mirrored_strategy.scope():
    model = get_model(tf_transform_output=tf_transform_output)

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps)

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(model,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Writing trainer.py
This cell will be skipped during export to pipeline.


In [26]:
# NOTE: Adjust the number of training and evaluation steps
TRAINING_STEPS = 10
EVALUATION_STEPS = 10

trainer = Trainer(
    module_file=os.path.abspath("trainer.py"),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

Train for 10 steps, validate for 10 steps
10/10 [==============================] - 56s 6s/step - loss: 0.6819 - accuracy: 0.5781 - val_loss: 0.6263 - val_accuracy: 0.6687
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: LIVE_VARS_IN
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: LIVE_VARS_IN
value: "\n\016\n\014asset_path:0\022\tvocab.txt"

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Trainer/model/6/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Trainer/model/6, id: 8)]
        ))

In [27]:
model_resolver = ResolverNode(
    instance_name='latest_blessed_model_resolver',
    resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing))

context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

## TensorFlow Model Evaluation

In [28]:
import apache_beam as beam
beam.__version__

'2.17.0'

In [29]:
# eval_config = tfma.EvalConfig(
#         model_specs=[tfma.ModelSpec(label_key='label')],
#         slicing_specs=[tfma.SlicingSpec()],
#         metrics_specs=tfma.metrics.default_binary_classification_specs()
# )

# model_analyzer = Evaluator(
#     examples=example_gen.outputs['examples'],
#     model=trainer.outputs['model'],
#     baseline_model=model_resolver.outputs['model'],
#     eval_config=eval_config)

# context.run(model_analyzer)

import tensorflow_model_analysis as tfma
from tfx.components.evaluator.component import Evaluator

model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='label')],
        slicing_specs=[tfma.SlicingSpec(), ],
        metrics_specs=tfma.metrics.default_binary_classification_specs()
        ))
context.run(model_analyzer)

TypeError: __init__() got an unexpected keyword argument 'dtype'

In [30]:
%%skip_for_export

model_analyzer.outputs

{'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Evaluator/evaluation/8, id: 9)]
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-04-20T00_50_22.602184-u42clphw/Evaluator/blessing/8, id: 10)]
)}

This cell will be skipped during export to pipeline.


In [31]:
%%skip_for_export

blessing_uri = model_analyzer.outputs.blessing.get()[0].uri
!ls -l {blessing_uri}

OSError: [Errno 12] Cannot allocate memory

This cell will be skipped during export to pipeline.


## Model Export for Serving

In [32]:
!mkdir -p ./content/serving_model_dir

serving_model_dir = './content/serving_model_dir'

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_dir)))

context.run(pusher)

OSError: [Errno 12] Cannot allocate memory

In [33]:
%%skip_for_export

pusher.outputs

NameError: name 'pusher' is not defined

This cell will be skipped during export to pipeline.


In [34]:
ls -al /tmp/tfx-interactive-2020-03-31T23_47_21.775461-e63ksgi7/Pusher/pushed_model/9

OSError: [Errno 12] Cannot allocate memory

In [35]:
%%skip_for_export

push_uri = pusher.outputs.model_push.get()[0].uri

print(os.listdir(push_uri))
latest_version = max(os.listdir(push_uri))
latest_version_path = os.path.join(push_uri, latest_version)
model = tf.saved_model.load(latest_version_path)

for item in model.signatures.items():
    pp.pprint(item)

NameError: name 'pusher' is not defined

This cell will be skipped during export to pipeline.


## Upload the Exported Model to GDrive

In [ ]:
!zip -r ./content/export_model.zip {pusher.outputs.model_push.get()[0].uri}

In [ ]:
# """
# The way to upload the zip file with the exported model was suggested in this 
# Stack Overflow answer https://stackoverflow.com/questions/50453428/how-do-i-download-multiple-files-or-an-entire-folder-from-google-colab
# """

# from google.colab import files
# from google.colab import auth
# from googleapiclient.http import MediaFileUpload
# from googleapiclient.discovery import build

# zip_file = "export_model.zip"

# def save_file_to_drive(name, path):
#     file_metadata = {
#     'name': name,
#     'mimeType': 'application/octet-stream'
#     }

#     media = MediaFileUpload(path, 
#                   mimetype='application/octet-stream',
#                   resumable=True)

#     created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

#     print('File ID: {}'.format(created.get('id')))

#     return created

# auth.authenticate_user()
# drive_service = build('drive', 'v3')

# destination_name = zip_file
# path_to_file = zip_file
# save_file_to_drive(destination_name, path_to_file)

## Test the Exported Model

* Load the model as a tf.ConcreteFunction and predict with a test sample 


#### Test by loading the model as a tf.ConcreteFunction

In [ ]:
loaded_model = tf.saved_model.load(latest_version_path)
f = loaded_model.signatures["serving_default"]
print(f(tf.constant([[b"This is a test"]])))